In [28]:
"""members scheduling problem with shift requests."""
from ortools.sat.python import cp_model
import pandas as pd
from IPython.display import display

def main():
    # This program tries to find an optimal assignment of members to shifts
    # (3 shifts per day, for 7 days), subject to some constraints (see below).
    # Each member can request to be assigned to specific shifts.
    # The optimal assignment maximizes the number of fulfilled shift requests.
    num_members = 50
    num_shifts = 4
    num_days = 26
    all_members = range(num_members)
    all_shifts = range(num_shifts)
    all_days = range(num_days)
    # shift_requests = [[[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
    #                    [0, 1, 0], [0, 0, 1]],
    #                   [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0],
    #                    [0, 0, 0], [0, 0, 1]],
    #                   [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0],
    #                    [0, 1, 0], [0, 0, 0]],
    #                   [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0],
    #                    [1, 0, 0], [0, 0, 0]],
    #                   [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0],
    #                    [0, 1, 0], [0, 0, 0]]]

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: member 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_members:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    # Each shift is assigned to exactly one member in .
    for d in all_days:
        for s in all_shifts:
            model.Add(sum(shifts[(n, d, s)] for n in all_members) == 1)

    # Each member works at most one shift per day.
    for n in all_members:
        for d in all_days:
            model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

    # Try to distribute the shifts evenly, so that each member works
    # min_shifts_per_member shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of members, some members will
    # be assigned one more shift.
    min_shifts_per_member = (num_shifts * num_days) // num_members
    if num_shifts * num_days % num_members == 0:
        max_shifts_per_member = min_shifts_per_member
    else:
        max_shifts_per_member = min_shifts_per_member + 1
    for n in all_members:
        num_shifts_worked = 0
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked += shifts[(n, d, s)]
        model.Add(min_shifts_per_member <= num_shifts_worked)
        model.Add(num_shifts_worked <= max_shifts_per_member)

    # pylint: disable=g-complex-comprehension
    # model.Maximize(
    #     sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_members
    #         for d in all_days for s in all_shifts))

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    days = {}

    if status == cp_model.OPTIMAL:
        print('Solution:')
        # print(solver.Value(shifts[(1, 2, 1)]) == 1)
        for d in all_days:
            days[d] = {}
            for m in all_members:
                days[d][m] = "-"
                for s in all_shifts:
                    if solver.Value(shifts[(m, d, s)]) == 1:
                        # if shift_requests[m][d][s] == 1:
                        #     print('Member', m, 'works shift', s, '(requested).')
                        # else:
                        days[d][m] = s

                        # print('Member', m, 'works shift', s,
                        #       '(not requested).')
        print(f'Number of shift requests met = {solver.ObjectiveValue()}',
              f'(out of {num_members * min_shifts_per_member})')
    else:
        print('No optimal solution found !')
    
    df = pd.DataFrame(days)
    display(df)

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())


if __name__ == '__main__':
    main()

Solution:
Number of shift requests met = 0.0 (out of 100)


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,2,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,-,-,-,-,-,2,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,-,-,0,-,0,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,0,-,0,-
5,-,-,-,-,-,-,2,-,-,-,...,-,-,-,-,-,-,-,-,-,2
6,-,3,3,-,1,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,-,-,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,-,-,-,-,-,-,0,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,2,1,-,-



Statistics
  - conflicts: 0
  - branches : 0
  - wall time: 0.684435 s
